In [1]:
import pickle

In [2]:
import glob
import errno
import unicodedata

In [ ]:
testing = "1928_in_association_football 0 The following are the football -LRB- soccer -RRB- events of the year 1928 throughout the world ."
testing1 = testing.split()
print(testing1)
testing2 = " ".join(testing1)
print(testing2)

# Preprosessing 

In [3]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download('wordnet')
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


sw = stopwords.words('english') + ['!',',','.','?','\s',"\n","-lrb-","-rrb-"]
def pre_process(sentence):
    tokens = nltk.word_tokenize(sentence)
    tokens = [lemmatize(t) for t in tokens]
    tokens = [t.lower() for t in tokens]
    tokens =  [t for t in tokens if t not in sw]
    return tokens

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\NAFIS\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\NAFIS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# All wiki  pages into a dict of dict

In [34]:
path = 'wiki-pages-text/*'
files = glob.glob(path)
#print(files[0])
#name = files[0]
count = 0
faultySentenceIdCount = 0
wikiShards = {}

for name in files:
    count += 1
    print(count)
    with open(name, encoding="utf8") as f:
        for line in f:
            eachLine = line.split()
            docId = unicodedata.normalize('NFD', eachLine[0])
            tempSentenceId = eachLine[1]
            if tempSentenceId.isdigit():
                #continue
                sentenceId = int(tempSentenceId)
                sentence = " ".join(eachLine[2:])
                #newSentence = {sentenceId:sentence}
                
                if docId in wikiShards:
                    
                    wikiShards[docId][sentenceId] = sentence
                    
                else:
                    
                    wikiShards[docId] = {}
                    wikiShards[docId][sentenceId] = sentence
            else:
                faultySentenceIdCount += 1


with open('all_wiki_as_dict.pickle', 'wb') as handle:
    pickle.dump(wikiShards, handle, protocol=pickle.HIGHEST_PROTOCOL)            
print(count)
print(faultySentenceIdCount)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
109
499


In [35]:
with open('all_wiki_as_dict.pickle', 'rb') as handle:
    allWikiDocs = pickle.load(handle)

print("done")
print(len(allWikiDocs))

done
5396106


#  Extracting keywords from claims

In [5]:
import json
all_claims_keywords = set()
with open('devset.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))

with open('train.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))

with open('test-unlabelled.json', 'r') as f:
    info = json.load(f)

for key in info:
    all_claims_keywords.update(pre_process(info[key]["claim"]))

print(len(all_claims_keywords))

with open('all_claims_keywords.pickle', 'wb') as handle:
    pickle.dump(all_claims_keywords, handle, protocol=pickle.HIGHEST_PROTOCOL)

4411
21050
22548


In [36]:
for index in allWikiDocs['1986_NBA_Finals']:
    print(allWikiDocs['1986_NBA_Finals'][index])

The 1986 NBA Finals was the championship round of the 1985 -- 86 NBA season .
It pitted the Eastern Conference champion Boston Celtics against the Western Conference champion Houston Rockets , in a rematch of the 1981 Finals -LRB- only Allen Leavell and Robert Reid remained from the Rockets ' 1981 team -RRB- .
The Celtics defeated the Rockets four games to two to win their 16th NBA championship .
The championship would be the Celtics ' last until the 2008 NBA Finals .
Larry Bird was named the Finals MVP .
On another note , this series marked the first time the `` NBA Finals '' branding was officially used , as they dropped the `` NBA World Championship Series '' branding which had been in use since the beginning of the league , though it had been unofficially called the `` NBA Finals '' for years .
Until the 2011 series , this was the last time the NBA Finals had started before June .
Since game three , all NBA Finals games have been played in June .
Starting with the following year , 

In [26]:
all_docs_of_interest = {}
for key in allWikiDocs:
    all_words_in_doc = set()
    for index in allWikiDocs[key]:
        all_words_in_doc.update(pre_process(allWikiDocs[key][index]))
        
    if not all_words_in_doc.isdisjoint(all_claims_keywords):
        all_docs_of_interest[key] = allWikiDocs[key]

In [27]:
print(len(all_docs_of_interest))

5395100


In [37]:
print(allWikiDocs['1986_NBA_Finals'])

{0: 'The 1986 NBA Finals was the championship round of the 1985 -- 86 NBA season .', 1: "It pitted the Eastern Conference champion Boston Celtics against the Western Conference champion Houston Rockets , in a rematch of the 1981 Finals -LRB- only Allen Leavell and Robert Reid remained from the Rockets ' 1981 team -RRB- .", 2: 'The Celtics defeated the Rockets four games to two to win their 16th NBA championship .', 3: "The championship would be the Celtics ' last until the 2008 NBA Finals .", 4: 'Larry Bird was named the Finals MVP .', 7: "On another note , this series marked the first time the `` NBA Finals '' branding was officially used , as they dropped the `` NBA World Championship Series '' branding which had been in use since the beginning of the league , though it had been unofficially called the `` NBA Finals '' for years .", 10: 'Until the 2011 series , this was the last time the NBA Finals had started before June .', 11: 'Since game three , all NBA Finals games have been pla

In [ ]:
print(all_docs_of_interest['1986_NBA_Finals'])

In [32]:
print(sw)
print(all_claims_keywords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
listKeys = list(allWikiDocs.keys())

docId_int_dict = {}

i = 0
for key in listKeys:
    docId_int_dict[key] = i
    i += 1

with open('docIdToIntDict.pickle', 'wb') as handle:
    pickle.dump(docId_int_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
with open('docIdToIntDict.pickle', 'rb') as handle:
    docIdIntDict = pickle.load(handle)
    
#print(docId_int_dict == docIdIntDict)

In [ ]:
import json

with open('train.json', 'r') as f:
    trainJsonDict = json.load(f)
    
print(len(trainJsonDict.keys()))

In [ ]:
trainingWikiDocs = {}
for key in trainJsonDict.keys():
    tempEvidenceList = trainJsonDict[key]['evidence']  
    for evidence in tempEvidenceList:
        foundDoc = unicodedata.normalize('NFD', evidence[0])
        if (foundDoc in allWikiDocs) and (foundDoc not in trainingWikiDocs):
            trainingWikiDocs[foundDoc] = allWikiDocs[foundDoc]
            
print("done")

In [ ]:
with open('allTrainShards.pickle', 'wb') as handle:
    pickle.dump(trainingWikiDocs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('allTrainShards.pickle', 'rb') as handle:
    allTrainDocs = pickle.load(handle)
    
#print(allTrainDocs == trainingWikiDocs)

In [ ]:
count = 0
for key in allTrainDocs.keys():
    print (key," ",allTrainDocs[key])
    count += 1
    print()
    if count > 10:
        break

In [ ]:
sentenceList = allTrainDocs['Nikolaj_Coster-Waldau']
#for sentence in sentenceList:
#    rawSentence = sentence.values()
totalSentence = []

sentence = sentenceList[0]
rawSentence = list(sentence.values())[0]
totalSentence.append(rawSentence)


sentence2 = sentenceList[1]
rawSentence2 = list(sentence2.values())[0]
totalSentence.append(rawSentence2)


#totalSentence = rawSentence+" "+rawSentence2

sentence3 = sentenceList[2]
rawSentence3 = list(sentence3.values())[0]

totalSentence.append(rawSentence3)

finalRawDoc = " ".join(totalSentence)
print(finalRawDoc)

In [ ]:
import gensim

In [ ]:
i = 0
docToIntMap = {}
justDocumentList = []
for doc in allWikiDocs.keys():
    sentenceList = allWikiDocs[doc]
    rawSentenceList = []
    for sentence in sentenceList:
        rawSentence = list(sentence.values())[0]
        rawSentenceList.append(rawSentence)
    finalRawDoc = " ".join(rawSentenceList)
    docToIntMap[doc] = i
    justDocumentList.append(finalRawDoc)   
    i += 1

print(type(justDocumentList))

In [ ]:
with open('justDocumentList.pickle', 'wb') as handle:
    pickle.dump(justDocumentList, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('docToIntMap.pickle', 'wb') as handle:
    pickle.dump(docToIntMap, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open('justDocumentList.pickle', 'rb') as handle:
    justDocumentList = pickle.load(handle)

In [ ]:
print(len(justDocumentList))

In [ ]:
with open('docToIntMap.pickle', 'rb') as handle:
    docToIntMap = pickle.load(handle)

In [ ]:
from nltk.tokenize import word_tokenize
gen_docs = [[w.lower() for w in word_tokenize(text)] 
            for text in justDocumentList]
#print(gen_docs)

In [ ]:
gen_docs = [[w for w in text.split()] 
            for text in justDocumentList]
#print(gen_docs)

In [ ]:
dictionary = gensim.corpora.Dictionary(gen_docs)

In [ ]:
corpus = [dictionary.doc2bow(gen_doc) for gen_doc in gen_docs]

In [ ]:
with open('corpusTemp.pickle', 'wb') as handle:
    pickle.dump(corpus, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
tf_idf = gensim.models.TfidfModel(corpus)

In [ ]:
s = 0
for i in corpus:
    s += len(i)
print(s)

In [ ]:
sims = gensim.similarities.Similarity('E:/(4) Semester 1 February 2019/COMP90042 Web Search and Text Analysis/Project Our/',tf_idf[corpus],
                                      num_features=len(dictionary))

In [ ]:
query_doc = [w.lower() for w in word_tokenize("The Ten Commandments is an epic film.")]
#print(query_doc)
query_doc_bow = dictionary.doc2bow(query_doc)
#print(query_doc_bow)
query_doc_tf_idf = tf_idf[query_doc_bow]
#print(query_doc_tf_idf)

In [ ]:
print(sims)
print(type(sims))

In [ ]:
print(type(sims[query_doc_tf_idf]))

In [ ]:
import numpy as np
#mylist = [3,2,1,5,0]
ind = np.argmax(sims[query_doc_tf_idf])
print(ind)



In [ ]:
print(list(docToIntMap.keys())[list(docToIntMap.values()).index(ind)])